In [16]:
import pandas as pd
import matplotlib.pyplot  as plt
import numpy as np
import plotly.express as px

# Importando Excel de defeitos e transformando para Dataframe Pandas

In [17]:
dados = pd.read_excel('../dados/SetoutDefect_20250103_142537_734275983 (1).xlsx', nrows=38625)
df = pd.DataFrame(dados)

In [18]:
df.head()

,Fábrica,Fabrica – Linha,Linha,Máquina de produção,Ordem de trabalhos,Ref. do artigo,Artigo,Ordem de sub-trabalho,Produto final,Lote,...,Nqa,Estado,A verificar,Comentário,Data de criação,Chave primária,Definir dia,Dia de criação,Turno,Definir chave primária
0,Wheaton,Wheaton - B1,B1,Máquina IS,193112,C1164.0000R,MB -1527-N1 | F | NH | C | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-01-01 06:03:07,2517600,2024-01-01,2024-01-01,1,872218
1,Wheaton,Wheaton - B1,B1,Máquina IS,193112,C1164.0000R,MB -1527-N1 | F | NH | C | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-01-01 06:03:07,2517601,2024-01-01,2024-01-01,1,872218
2,Wheaton,Wheaton - B1,B1,Máquina IS,193112,C1164.0000R,MB -1527-N1 | F | NH | C | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-01-01 06:57:09,2517738,2024-01-01,2024-01-01,1,872263
3,Wheaton,Wheaton - B1,B1,Máquina IS,193112,C1164.0000R,MB -1527-N1 | F | NH | C | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-01-01 06:57:09,2517739,2024-01-01,2024-01-01,1,872263
4,Wheaton,Wheaton - B1,B1,Máquina IS,193112,C1164.0000R,MB -1527-N1 | F | NH | C | LISO,NaN,NaN,NaN,...,0.0,0,0,NaN,2024-01-01 06:57:09,2517740,2024-01-01,2024-01-01,1,872263


# Criando novo Dataframe para trabalhar com os defeitos por data

### Criando coluna ``Data``, somente com data, sem o horário do registro

In [57]:
df_por_data = df.copy()

df_por_data['Data'] = df_por_data['Data de criação'].dt.date

df_por_data.groupby(['Ordem de trabalhos', 'Data'])[['Defeito']].count()

Defeito
Ordem de trabalhos Data               
192970             2024-01-03      162
                   2024-01-04      249
                   2024-01-05      268
                   2024-01-06      242
                   2024-01-07      232
...                                ...
194121             2024-06-27      218
                   2024-06-28      218
                   2024-06-29      213
                   2024-06-30      256
                   2024-07-01        1

[211 rows x 1 columns]

### Criando Dataframe multiíndice, agrupado por `Ordem de trabalhos`, `Data` e `Defeito`, contando a frequência de cada defeito 

In [56]:
frequencia_defeitos = df_por_data.groupby(['Ordem de trabalhos', 'Data', 'Defeito']).size().reset_index(name='Quantidade')
frequencia_defeitos = frequencia_defeitos.set_index(['Ordem de trabalhos', 'Data', 'Defeito'])
frequencia_defeitos = frequencia_defeitos.sort_index()

frequencia_defeitos

Quantidade
Ordem de trabalhos Data       Defeito                              
192970             2024-01-03 Cava no ombro                      10
                              Corpo torto                         6
                              Costura                             3
                              Costura desencontrada               7
                              Dobra                              10
...                                                             ...
194121             2024-06-30 Sujo de óleo                       58
                              Terminação deformada                1
                              Terminação suja                     5
                              Trinca por choque térmico          17
                   2024-07-01 Bolha                               1

[4687 rows x 1 columns]

# Criando um Dataframe resumo de defeitos

### Calculando o total de defeitos por cada ``Ordem`` e qual o defeito mais frequente de cada ``Ordem``

In [70]:
total_defeitos_por_ordem = frequencia_defeitos.groupby(level='Ordem de trabalhos')['Quantidade'].sum()

defeito_mais_frequente_por_ordem = (
    frequencia_defeitos.groupby(level='Ordem de trabalhos')['Quantidade'].idxmax().map(lambda x: x[2])
)

resumo_defeitos = pd.DataFrame({
    'Total de Defeitos': total_defeitos_por_ordem,
    'Defeito Mais Frequente': defeito_mais_frequente_por_ordem
})

# Trabalhando com Dataframe de medidas da gota e Dataframe de defeitos 

### Importando Dataframe com as medidas de cada ordem

In [67]:
medidas_ordem = pd.read_excel('../dados/media_ordem.xlsx', index_col=0)

### Unindo os dois Dataframes com o mesmo índice (`Ordem de prod`)

In [68]:
df_combinado = medidas_ordem.join(resumo_defeitos)
df_combinado

,Gotas cortadas (toneladas),Peso médio (g),Cortes por minuto,Comprimento médio (mm),Diâmetro,Temperatura (F°),Tras-PV (F°),Fritas-PV (F°),Agit-PV (F°),Meio-PV (F°),Frente-PV (F°),Total de Defeitos,Defeito Mais Frequente
Ordem de prod,,,,,,,,,,,,,
192970,19.865,169.91,87.33,125.51,30.97,2662.56,2348.88,2300.30,2250.90,2208.07,2176.10,1941,Sujo de óleo
193112,14.911,168.17,90.00,88.73,38.57,2205.86,2327.03,2307.63,2272.97,2246.77,2196.60,422,Sujo de óleo
193164,19.677,147.47,95.24,100.12,35.43,1744.15,2350.00,2299.89,2246.67,2201.89,2173.90,2574,Sujo de óleo
193206,18.578,171.15,89.35,90.40,36.88,2284.43,2315.75,2322.15,2280.57,2258.52,2179.75,927,Sujo de óleo
193207,13.691,170.87,59.98,98.23,36.56,2238.91,2348.35,2329.33,2298.58,2247.09,2160.38,1953,Sujo de óleo
193208,9.872,122.10,82.40,61.00,27.45,2179.31,2350.00,2306.35,2287.45,2257.50,2200.90,219,Sujo de óleo
193209,17.402,191.36,78.00,44.37,12.19,2012.13,2342.77,2294.00,2254.57,2231.21,2164.76,1663,Sujo de óleo
193342,20.438,169.96,88.08,114.87,33.44,1895.29,2338.96,2301.08,2241.66,2211.40,2171.24,3568,Sujo de óleo
193368,21.095,171.59,99.04,77.51,33.83,2077.62,2314.71,2292.40,2259.54,2231.44,2182.84,1581,Sujo de óleo
